In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sklearn
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from sklearn import svm
import scipy.io
import pickle

In [139]:
x = np.array([[1,2,3],
     [4,5,6],
     [7,8,9],
     [10, 11, 12]])

new_array = np.empty(np.shape(x))
for i in range(np.shape(x)[1]):
    print(x[:,i])
    print(normalize(x[:,i]))
    new_array[:,i] = normalize(x[:,i])
new_array


[ 1  4  7 10]
[ 0.04545455  0.18181818  0.31818182  0.45454545]
[ 2  5  8 11]
[ 0.07692308  0.19230769  0.30769231  0.42307692]
[ 3  6  9 12]
[ 0.1  0.2  0.3  0.4]


array([[ 0.04545455,  0.07692308,  0.1       ],
       [ 0.18181818,  0.19230769,  0.2       ],
       [ 0.31818182,  0.30769231,  0.3       ],
       [ 0.45454545,  0.42307692,  0.4       ]])

0
[ 1  4  7 10]
1
[ 2  5  8 11]
2
[ 3  6  9 12]


array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [1, 1, 1]])

In [171]:
def parse_mat_matrix(filename, category, normalization=False):
    f = open(filename, 'r')
    lines = f.readlines()
    read = False
    for indx, line in enumerate(lines):
        if ';' in line:
            read = False
        if read:
            vector = line.split(' ')
            # parse out %
            comment_indx = vector.index('%')
            vector = vector[:comment_indx]
            matrix = np.vstack((matrix, vector))
        if category in line:
            read = True
            vector = lines[indx+1].split(' ')
            comment_indx = vector.index('%')
            length = len(vector[:comment_indx])
            matrix = np.empty(length)
    matrix = np.array(matrix[1:], dtype='float64')    
    
    if normalization:
        new_matrix = np.empty(np.shape(matrix))
        for i in range(np.shape(matrix)[1]):
            new_matrix[:,i] = normalize(matrix[:,i])
        return new_matrix[:-2]
    # no lost and total
    return matrix[:-2]


def parse_mat_name(filename):
    f = open(filename, 'r')
    lines = f.readlines()
    read = False
    names = []
    for line in lines:
        if ';' in line:
            read = False
        if read:
            iso = line.replace(' ', '').replace("'", '').strip()
            names.append(iso)
        if 'NAMES' in line:
            read = True
    return names[:-2]

def parse_mat_burnup(filename):
    f = open(filename, 'r')
    lines = f.readlines()
    for line in lines:
        if 'TOT_BURNUP' in line:
            bu = line.split()
            a = bu.index('[') + 1
            z = bu.index('];')
            bu = bu[a:z]
    return np.array(bu, dtype='float64') * 1e3

def name_and_matrix_to_dict(name_list, matrix):
    return_dict = {}
    for indx, val in enumerate(name_list):
        return_dict[val] = matrix[indx]
    return return_dict

def normalize(array):
    new_array = np.empty(len(array))
    for indx,val in enumerate(array):
        new_array[indx] = val / sum(array)
    return new_array

# def name_bu_matrix_to_df(name_list, bu_list, matrix):
    

In [172]:
mat = parse_mat_matrix('./serpent_test_burnup/case_1_enrichment_3.3%/pwr_dep.m', 'MAT_UO2_MDENS',
                       normalization=True)

In [173]:
name = parse_mat_name('./serpent_test_burnup/case_1_enrichment_3.3%/pwr_dep.m')

In [174]:
filename = './serpent_test_burnup/case_1_enrichment_3.3%/pwr_dep.m'
name_and_matrix_to_dict(name_list = parse_mat_name(filename),
                        matrix = parse_mat_matrix(filename, 'MAT_UO2_MDENS'))

{'H1': array([  0.00000000e+00,   2.88060000e-08,   3.04648000e-08,
          3.21088000e-08,   3.45472000e-08,   3.79990000e-08,
          3.86489000e-08,   3.86505000e-08,   3.86827000e-08,
          4.03594000e-08,   4.28146000e-08,   4.32009000e-08,
          4.43784000e-08,   4.60871000e-08,   5.07028000e-08,
          5.10102000e-08,   5.50312000e-08,   5.75869000e-08,
          6.90288000e-08,   6.90872000e-08,   6.92675000e-08]),
 'H2': array([  0.00000000e+00,   6.57853000e-09,   6.97100000e-09,
          7.35503000e-09,   7.92475000e-09,   8.73916000e-09,
          8.88785000e-09,   8.88821000e-09,   8.89562000e-09,
          9.28533000e-09,   9.86093000e-09,   9.95360000e-09,
          1.02275000e-08,   1.06305000e-08,   1.16221000e-08,
          1.16930000e-08,   1.26576000e-08,   1.32305000e-08,
          1.59258000e-08,   1.59393000e-08,   1.59808000e-08]),
 'H3': array([  0.00000000e+00,   5.86186000e-13,   6.44253000e-13,
          7.00506000e-13,   7.90740000e-13,   9.

In [175]:
# do it for all dep files
import os
subdirectory_list = [x[0] for x in os.walk('./serpent_test_burnup')][1:]
dir_dict = {}
for subdir in subdirectory_list:
    enr = subdir.split('_')[-1]
    enr = enr.replace('%', '')
    dir_dict[enr] =  subdir
dir_dict

{'3.3': './serpent_test_burnup/case_1_enrichment_3.3%',
 '4.9': './serpent_test_burnup/case_10_enrichment_4.9%',
 '4.1': './serpent_test_burnup/case_6_enrichment_4.1%',
 '3.2': './serpent_test_burnup/case_7_enrichment_3.2%',
 '4.7': './serpent_test_burnup/case_5_enrichment_4.7%',
 '4.6': './serpent_test_burnup/case_2_enrichment_4.6%',
 '3.5': './serpent_test_burnup/case_9_enrichment_3.5%',
 '3.9': './serpent_test_burnup/case_3_enrichment_3.9%',
 '3.7': './serpent_test_burnup/case_4_enrichment_3.7%',
 '3.6': './serpent_test_burnup/case_8_enrichment_3.6%'}

In [176]:
coll_dict = {}
first = True
for enr, path in dir_dict.items():
    filename = path + '/pwr_dep.m'
    name_matrix_dict = name_and_matrix_to_dict(name_list = parse_mat_name(filename),
                        matrix = parse_mat_matrix(filename, 'MAT_UO2_MDENS', normalization=True))
    name_matrix_dict['bu'] = parse_mat_burnup(filename)
    name_matrix_dict['enr'] = [enr] * len(bu)
    coll_dict[enr] = name_matrix_dict

In [177]:
enr_num = len(dir_dict.keys())
bu_num = len(parse_mat_burnup(filename))
print('TOTAL NUMBER OF DEPLETION CALCULATIONS SHOULD BE: \n'
      '[enrichment variation * burnup steps]\n'
      '%i * %i = %i'
%(enr_num, bu_num, enr_num * bu_num))

TOTAL NUMBER OF DEPLETION CALCULATIONS SHOULD BE: 
[enrichment variation * burnup steps]
10 * 21 = 210


In [178]:
df_dict = {}
first_key = list(coll_dict.keys())[0]
for category in coll_dict[first_key].keys():
    long_vec = np.empty(0)
    for key, val in coll_dict.items():
        long_vec = np.append(long_vec, val[category])
    df_dict[category] = long_vec

In [179]:
df = pd.DataFrame.from_dict(df_dict)

In [180]:
df

,H1,H2,H3,He3,He4,Li6,Li7,Be9,B10,B11,...,Cm239,Cm251,Bk249,Bk250,Bk251,Cf249,Cf250,Cf251,bu,enr
0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,3.3
1,1.377841e-09,3.146625e-10,2.803829e-14,4.816703e-16,0.000002,4.408144e-18,1.539067e-21,8.871782e-14,1.650644e-25,3.370133e-17,...,6.148482e-25,2.384512e-22,8.340754e-12,1.231299e-25,7.888122e-22,2.006025e-12,7.085458e-23,1.596504e-18,43809.2,3.3
2,1.457197e-09,3.334380e-10,3.081601e-14,5.041223e-16,0.000002,4.985690e-18,1.823329e-21,9.829462e-14,1.834851e-25,3.758503e-17,...,1.390926e-24,5.166448e-22,1.273330e-11,2.054894e-25,1.708732e-21,2.894243e-12,1.044123e-22,2.577967e-18,45376.8,3.3
3,1.535848e-09,3.518103e-10,3.350703e-14,5.228487e-16,0.000002,5.626454e-18,2.136434e-21,1.087551e-13,2.006894e-25,4.135105e-17,...,1.483246e-24,8.349612e-22,1.895922e-11,3.239549e-25,2.761702e-21,4.239547e-12,1.589932e-22,4.213244e-18,46954.1,3.3
4,1.652502e-09,3.790658e-10,3.782359e-14,5.569549e-16,0.000002,6.670668e-18,2.662016e-21,1.252382e-13,2.327341e-25,4.766612e-17,...,1.742112e-24,1.530250e-21,3.235205e-11,5.873481e-25,5.061705e-21,7.370563e-12,2.939414e-22,8.300344e-18,49242.1,3.3
5,1.817643e-09,4.180288e-10,4.428398e-14,6.176132e-16,0.000002,8.308571e-18,3.538098e-21,1.500968e-13,2.895027e-25,5.760599e-17,...,2.084069e-24,3.247420e-21,6.332071e-11,1.218265e-24,1.074227e-20,1.515682e-11,6.500877e-22,1.934392e-17,52327.6,3.3
6,1.848733e-09,4.251419e-10,4.547349e-14,6.287690e-16,0.000002,8.631367e-18,3.715395e-21,1.548943e-13,3.010288e-25,5.947255e-17,...,2.129266e-24,3.719987e-21,7.103022e-11,1.377147e-24,1.230555e-20,1.717974e-11,7.450440e-22,2.232922e-17,52881.9,3.3
7,1.848810e-09,4.251590e-10,4.547640e-14,6.287881e-16,0.000002,8.632179e-18,3.715830e-21,1.549062e-13,3.010541e-25,5.947685e-17,...,2.115365e-24,3.728472e-21,7.105078e-11,1.377568e-24,1.231755e-20,1.718510e-11,7.452975e-22,2.233716e-17,52883.3,3.3
8,1.850350e-09,4.255136e-10,4.553792e-14,6.292044e-16,0.000002,8.649306e-18,3.724857e-21,1.551560e-13,3.015722e-25,5.957396e-17,...,2.082752e-24,3.744373e-21,7.147077e-11,1.386307e-24,1.238615e-20,1.729670e-11,7.505593e-22,2.250253e-17,52912.1,3.3
9,1.930569e-09,4.441586e-10,4.872608e-14,6.557911e-16,0.000002,9.559570e-18,4.217979e-21,1.683768e-13,3.327498e-25,6.467217e-17,...,2.177553e-24,5.158562e-21,9.623190e-11,1.903280e-24,1.706470e-20,2.400629e-11,1.070090e-21,3.258688e-17,54392.8,3.3
